In [6]:
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io



from mmengine.config import Config
from mmselfsup.registry import DATA_SAMPLERS, DATASETS, MODELS, TRANSFORMS
from mmselfsup.models.utils import Extractor

from mmengine.dist import get_rank, init_dist
from mmengine.registry import init_default_scope

from mmengine.runner import Runner, load_checkpoint
from torch.utils.data import DataLoader

from mmengine.dataset import pseudo_collate, worker_init_fn, default_collate

cfg_pth = '/om2/user/amarvi/mmselfsup/work_dirs/selfsup/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj.py'
ckpt_pth = '/om2/user/amarvi/mmselfsup/work_dirs/selfsup/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-obj/epoch_100.pth'

cfg = Config.fromfile(cfg_pth)

init_default_scope(cfg.get('default_scope', 'mmselfsup'))

In [40]:
which = 'face'
cfg_pth = '/om2/user/amarvi/mmselfsup/work_dirs/selfsup/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-%s/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-%s.py'%(which,which)
ckpt_pth = '/om2/user/amarvi/mmselfsup/work_dirs/selfsup/dino_vit-base-p16_8xb64-amp-coslr-100e_mini-%s/epoch_100.pth'%which

cfg = Config.fromfile(cfg_pth)

cfg.model
cfg.model.backbone.out_indices = [0,1,2,3,4,5,6,7,8,9,10]

model = MODELS.build(cfg.model)
model.init_weights()
load_checkpoint(model, ckpt_pth, map_location='cpu')

model = model.cuda()

08/04 15:34:20 - mmengine - INFO - 
backbone.cls_token - torch.Size([1, 1, 768]): 
The value is the same before and after calling `init_weights` of DINO  
 
08/04 15:34:20 - mmengine - INFO - 
backbone.pos_embed - torch.Size([1, 197, 768]): 
Initialized by user-defined `init_weights` in VisionTransformer  
 
08/04 15:34:20 - mmengine - INFO - 
backbone.patch_embed.projection.weight - torch.Size([768, 3, 16, 16]): 
The value is the same before and after calling `init_weights` of DINO  
 
08/04 15:34:20 - mmengine - INFO - 
backbone.patch_embed.projection.bias - torch.Size([768]): 
The value is the same before and after calling `init_weights` of DINO  
 
08/04 15:34:20 - mmengine - INFO - 
backbone.layers.0.ln1.weight - torch.Size([768]): 
The value is the same before and after calling `init_weights` of DINO  
 
08/04 15:34:20 - mmengine - INFO - 
backbone.layers.0.ln1.bias - torch.Size([768]): 
The value is the same before and after calling `init_weights` of DINO  
 
08/04 15:34:20 - mm

In [41]:
dataset_type = 'mmcls.CustomDataset'
data_root = '/om2/user/amarvi/FACE/data/behav_test'
extract_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='mmcls.ResizeEdge', scale=256, edge='short'),
    dict(type='CenterCrop', crop_size=224),
    dict(type='PackSelfSupInputs')
]

train_dataloader = dict(
    batch_size=1,
    num_workers=2,
    dataset=dict(
        type=dataset_type,
        data_root=data_root,
        pipeline=extract_pipeline),
    collate_fn=dict(type='default_collate'),
    sampler=dict(type='DefaultSampler', shuffle=False),
)

# pooling cfg
# this isn't the right pool for ViT
# output of backbone is Tuple[List[Tensor]]]: (11, (2, [batch-size ,768, 14, 14]))
# 11 layers, 2 ???, batch-size, output of backbone, 14x14 patches?

# each layer outputs 2 tensors: [batch-size, 768, 14, 14] (image tokens) and [batch-size, 768] (clss token)
pool_cfg = dict(type='AvgPool2d_DINO')
# pool_cfg='test'

data_loader_train = Runner.build_dataloader(dataloader=train_dataloader)

extractor_train = Extractor(
        extract_dataloader=data_loader_train,
        dist_mode=False,
        pool_cfg=pool_cfg)

In [42]:
outputs = extractor_train(model)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 200/200, 45.9 task/s, elapsed: 4s, ETA:     0sfeat 1 200
feat 2 200
feat 3 200
feat 4 200
feat 5 200
feat 6 200
feat 7 200
feat 8 200
feat 9 200
feat 10 200
feat 11 200


In [43]:
outputs

{'feat 1': tensor([[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0'),
 'feat 2': tensor([[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0'),
 'feat 3': tensor([[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0'),
 'feat 4': tensor([[nan, 

In [19]:
df = pd.read_pickle('/om2/user/amarvi/FACE/saved_models/behav_mini_results_all.pkl')
df

,model,size,dataset,layer,results,accuracy,ci
0,supervised,100,_things2,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, ...",0.831410,"(0.8128205128205128, 0.8493589743589743)"
1,simclr,100,_things2,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...",0.796154,"(0.7756410256410257, 0.8153846153846154)"
2,supervised,100,_face,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.916026,"(0.9019230769230769, 0.9294871794871795)"
3,simclr,100,_face,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, ...",0.684615,"(0.6608974358974359, 0.7070512820512821)"
4,supervised,100,_car,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.851923,"(0.8346153846153846, 0.8692307692307693)"
5,simclr,100,_car,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",0.832692,"(0.8134615384615385, 0.8506410256410256)"
6,supervised,100,_dog,backbone.layer4.2.relu,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.751282,"(0.7294871794871794, 0.7717948717948718)"
7,simclr,100,_dog,backbone.layer4.2.relu,"[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, ...",0.626923,"(0.6025641025641025, 0.6506410256410257)"


In [7]:
f = scipy.io.loadmat('/om2/user/amarvi/FACE/data/data_up.mat')

triplet = f['data_up'][0][0][0]
perf = f['data_up'][0][0][1]

print(triplet.shape, perf.shape)

(3, 1560) (1, 1560)


In [30]:
act

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [28]:
for l, act in outputs.items():
    act = act.cpu().numpy()
    print(l, act.shape)

    count = 0
    bstrap = []
    
    for idx, trio in enumerate(triplet.transpose()):
        trio = trio-1
        gt = trio//5
        [targ, m1, m2] = trio

        dist1 = np.linalg.norm(act[targ] - act[m1])
        dist2 = np.linalg.norm(act[targ] - act[m2])
        model_choice = np.argmax(np.array([dist1, dist2])) + 1
        correct_choice = np.where(gt != gt[0])[0]

        bstrap.append(int(model_choice == correct_choice))
        if model_choice != correct_choice:
            count += 1

    bstrap = np.array(bstrap)
    n_bootstrap = 10000
    # Bootstrap resampling
    bootstrap_samples = np.random.choice(bstrap, size=(len(bstrap), n_bootstrap), replace=True)
    # Calculate the mean of each bootstrap sample
    bootstrap_sample_means = np.mean(bootstrap_samples, axis=0)
    # Calculate the 95% confidence interval using percentile method
    ci_lower = np.percentile(bootstrap_sample_means, 2.5)
    ci_upper = np.percentile(bootstrap_sample_means, 97.5)

    print("CI:", (ci_lower, ci_upper), 'Mean:', np.mean(bstrap))

    acc = 1 - count/len(triplet[0])
    # df.loc[len(df)] = {'model': 'dino', 'size': '100', 'dataset': '_face', 'layer': l, 'results': bstrap, 'accuracy': acc, 'ci': (ci_lower, ci_upper)}

feat 1 (200, 768)
CI: (0.4756410256410256, 0.5243589743589744) Mean: 0.5
feat 2 (200, 768)
CI: (0.4756410256410256, 0.525) Mean: 0.5
feat 3 (200, 768)
CI: (0.4756410256410256, 0.525) Mean: 0.5
feat 4 (200, 768)
CI: (0.475, 0.5243589743589744) Mean: 0.5
feat 5 (200, 768)
CI: (0.4756410256410256, 0.5243749999999998) Mean: 0.5
feat 6 (200, 768)
CI: (0.475, 0.525) Mean: 0.5
feat 7 (200, 768)
CI: (0.4756410256410256, 0.5243589743589744) Mean: 0.5
feat 8 (200, 768)
CI: (0.475, 0.5256410256410257) Mean: 0.5
feat 9 (200, 768)
CI: (0.47562499999999996, 0.525) Mean: 0.5
feat 10 (200, 768)
CI: (0.475, 0.5243589743589744) Mean: 0.5
feat 11 (200, 768)
CI: (0.475, 0.5243589743589744) Mean: 0.5


In [27]:
df

,model,size,dataset,layer,results,accuracy,ci
0,supervised,100,_things2,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, ...",0.831410,"(0.8128205128205128, 0.8493589743589743)"
1,simclr,100,_things2,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, ...",0.796154,"(0.7756410256410257, 0.8153846153846154)"
2,supervised,100,_face,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.916026,"(0.9019230769230769, 0.9294871794871795)"
3,simclr,100,_face,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, ...",0.684615,"(0.6608974358974359, 0.7070512820512821)"
4,supervised,100,_car,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.851923,"(0.8346153846153846, 0.8692307692307693)"
5,simclr,100,_car,backbone.layer4.2.relu,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, ...",0.832692,"(0.8134615384615385, 0.8506410256410256)"
6,supervised,100,_dog,backbone.layer4.2.relu,"[1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.751282,"(0.7294871794871794, 0.7717948717948718)"
7,simclr,100,_dog,backbone.layer4.2.relu,"[1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, ...",0.626923,"(0.6025641025641025, 0.6506410256410257)"
8,dino,100,things2,feat 11,"[0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, ...",0.536538,"(0.5115384615384615, 0.5615384615384615)"
9,dino,100,_face,feat 11,"[0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, ...",0.500000,"(0.475, 0.525)"


In [21]:
data = tuple([torch.randn(11, 768, 14, 14), torch.randn(768)] for _ in range(11))
image_tokens = tuple(layer_data[0] for layer_data in data)
image_tokens[0].squeeze().shape

avgpool = m = nn.AdaptiveAvgPool2d((1, 1))

outs = []
for x in image_tokens:
    x_ = avgpool(x)
    outs.append(x_)

flat_features = [feat.view(feat.size(0), -1) for feat in outs]
flat_features[0].shape

torch.Size([11, 768])

In [ ]:
init_default_scope(cfg.get('default_scope', 'mmselfsup'))
if cfg.env_cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

os.environ['RANK'] = '0'
os.environ['LOCAL_RANK'] = '0'
os.environ['WORLD_SIZE'] = '0'
os.environ['MASTER_ADDR'] = "127.0.0.1"
os.environ['MASTER_PORT'] = '29500'
os.environ['PORT'] = '29500'

init_dist('pytorch')